In [1]:
import time
from tqdm.notebook import trange, tqdm

import numpy as np

from matplotlib import pyplot as plt
import matplotlib

from astropy.table import Table
from astropy.io.votable import parse_single_table, from_table, writeto
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.time import Time
import pyvo as vo

%matplotlib inline

In [2]:
ztfobjects = parse_single_table("ztf_CXC_footprint_STAMP.xml").to_table()
skycoords = SkyCoord(ra=ztfobjects['meanra']*u.degree, dec=ztfobjects['meandec']*u.degree, frame='icrs')
# cone = vo.dal.SCSService('http://cda.cfa.harvard.edu/csc2scs/coneSearch')
cone = vo.dal.SCSService('https://cda.cfa.harvard.edu/cxcscs/coneSearch')
maxrad = 50.0 * u.arcmin

csc_results = []

# Start timer
start = time.time()

for i in trange(len(ztfobjects), desc='Indexing'):
    results = cone.search(pos=skycoords[i], radius=maxrad)
    csc_results.append(results)

# End timer
end = time.time()
print(f"Runtime of the program is {end - start}")

csc_res = np.asarray(csc_results, dtype=object)


Runtime of the program is 612.2139730453491


In [3]:
ztfobjects['results']=csc_res

f = np.vectorize(len)
ztfobjects['n_res']=f(ztfobjects['results'])

mask = (ztfobjects['n_res'] > 0)
maskedztf = ztfobjects[mask]
print('Length: %i' % len(maskedztf))
maskedztf[:5]

Length: 5994


oid,ndethist,ncovhist,mjdstarthist,mjdendhist,corrected,stellar,ndet,g_r_max,g_r_max_corr,g_r_mean,g_r_mean_corr,firstmjd,lastmjd,deltajd,meanra,meandec,sigmara,sigmadec,class,classifier,probability,step_id_corr,results,n_res
str12,str4,int64,float64,float64,bool,bool,int64,object,object,object,object,float64,float64,float64,float64,float64,object,object,str8,str16,float64,str16,object,int64
ZTF18abiwqjk,10,1759,58316.432106500026,59329.505381899886,True,True,2,None,None,None,None,58435.2652431,59329.505381899886,894.2401387998834,3.2398945,50.35809655,0.00012586500705121358,3.698168465613856e-05,bogus,stamp_classifier,0.38414162,correction_0.0.1,<Table length=1> obsid ra dec ... status public_avail_date deg deg ... object float64 float64 ... object object ------ ----------------- ----------------- ... -------- ------------------- 1864 3.072916666666667 50.43136111111111 ... archived 2002-01-09T00:00:00,1
ZTF18abiagdn,4,1770,58317.3802894,59329.505381899886,True,True,2,None,None,None,None,58435.2652431,59329.505381899886,894.2401387998834,3.23719265,50.10339005,0.0004404568140012446,0.00024515392103841997,bogus,stamp_classifier,0.6067099,correction_0.0.1,<Table length=1> obsid ra dec ... status public_avail_date deg deg ... object float64 float64 ... object object ------ ----------------- ----------------- ... -------- ------------------- 1864 3.072916666666667 50.43136111111111 ... archived 2002-01-09T00:00:00,1
ZTF18abiagbp,22,1592,58312.42182870023,59329.505381899886,True,True,11,-0.113813825,0.46666965,-0.23509538,0.553391,58435.2652431,59329.505381899886,894.2401387998834,3.092486972727272,50.5531224,0.0002128165176432332,9.867637001850209e-05,VS,stamp_classifier,0.73947304,correction_0.0.1,<Table length=1> obsid ra dec ... status public_avail_date deg deg ... object float64 float64 ... object object ------ ----------------- ----------------- ... -------- ------------------- 1864 3.072916666666667 50.43136111111111 ... archived 2002-01-09T00:00:00,1
ZTF18abfusmm,109,3560,58308.400335600134,59329.505381899886,True,True,58,-0.704526,0.158681,0.3384379,0.46228766,58427.186979200225,59329.505381899886,902.3184026996605,2.092935617241379,51.358773613793105,0.0001636388922779321,9.392789591505443e-05,VS,stamp_classifier,0.76604766,correction_0.0.1,<Table length=2> obsid ra dec ... status public_avail_date deg deg ... object float64 float64 ... object object ------ ------------------ ----------------- ... -------- ------------------- 976 2.070833 51.381667 ... archived 2001-12-08T00:00:00 8942 2.0766666666666667 51.38861111111111 ... archived 2009-03-20T00:00:00,2
ZTF18acsxgvk,50,918,58456.15988430008,59329.50489579979,True,True,39,1.126339,1.008395,0.9848154,0.9783188,58456.15988430008,59329.50489579979,873.3450114997104,354.40294933333337,29.929757820512823,0.00013856618280670832,8.93838033707826e-05,VS,stamp_classifier,0.38897365,correction_0.0.1,<Table length=2> obsid ra dec ... status public_avail_date deg deg ... object float64 float64 ... object object ------ ----------------- -------- ... -------- ------------------- 11353 354.415 30.12941 ... archived 2010-05-09T00:00:00 21279 355.0179166666667 30.2965 ... archived 2019-11-08T00:00:00,2


In [7]:
n = maskedztf['results'].data
obsids = np.asarray([t['obsid'].data.astype(str) for t in n], dtype=object)
of = np.hstack(obsids)
tof = tuple(of)
tof

('1864',
 '1864',
 '1864',
 '976',
 '8942',
 '11353',
 '21279',
 '14052',
 '3731',
 '9554',
 '10569',
 '4206',
 '4816',
 '17003',
 '7124',
 '10569',
 '4206',
 '11732',
 '19508',
 '20515',
 '23166',
 '20515',
 '23166',
 '7768',
 '20416',
 '9261',
 '9525',
 '12991',
 '9241',
 '9261',
 '1864',
 '976',
 '8942',
 '976',
 '8942',
 '1864',
 '1864',
 '5643',
 '11353',
 '21279',
 '14052',
 '2987',
 '20515',
 '23166',
 '7768',
 '10569',
 '4206',
 '9261',
 '1864',
 '976',
 '8942',
 '11353',
 '21279',
 '20416',
 '9525',
 '12991',
 '9241',
 '20617',
 '976',
 '8942',
 '1864',
 '9261',
 '16664',
 '11830',
 '17003',
 '7124',
 '3082',
 '22193',
 '7121',
 '7101',
 '16015',
 '17412',
 '7896',
 '16014',
 '7895',
 '802',
 '16015',
 '17412',
 '7896',
 '16014',
 '7895',
 '802',
 '844',
 '12373',
 '16015',
 '17412',
 '7896',
 '16014',
 '7895',
 '802',
 '844',
 '12373',
 '844',
 '12373',
 '17213',
 '976',
 '8942',
 '845',
 '23245',
 '23241',
 '21691',
 '23040',
 '24840',
 '1512',
 '1505',
 '211',
 '212',
 '215

In [8]:
tapservice = vo.dal.TAPService("https://cda.cfa.harvard.edu/cxctap/")
query = '''
SELECT o.obs_id, o.obs_creation_date, o.s_ra, o.s_dec, o.s_region
FROM ivoa.ObsCore AS o
WHERE o.dataproduct_type = 'event' AND o.obs_id IN {}
'''.format(tof)

# Start timer
start = time.time()

tapresult = tapservice.search(query)

# End timer
end = time.time()
print(f"Runtime of the program is {end - start}")

tresult = tapresult.to_table()
tresult['obs_creation_date'] = Time(tresult['obs_creation_date'].astype(str), format='isot', scale='utc').mjd
tresult['obs_id'] = tresult['obs_id'].astype(str)

Runtime of the program is 163.99752497673035


In [26]:
# Testing to see if this works...
a = obsids[4]
print(a)
print(np.count_nonzero(np.in1d(tresult['obs_id'], a) == True))
tresult[np.in1d(tresult['obs_id'], a)]

['11353' '21279']
2


<Table length=2>
obs_id ...
       ...
 str5  ...
------ ...
 11353 ...
 21279 ...

In [24]:
footprints = np.asarray([tresult[np.in1d(tresult['obs_id'], a)] for a in obsids], dtype=object)
maskedztf['footprints'] = footprints

In [25]:
maskedztf[:3]

oid,ndethist,ncovhist,mjdstarthist,mjdendhist,corrected,stellar,ndet,g_r_max,g_r_max_corr,g_r_mean,g_r_mean_corr,firstmjd,lastmjd,deltajd,meanra,meandec,sigmara,sigmadec,class,classifier,probability,step_id_corr,results,n_res,footprints
str12,str4,int64,float64,float64,bool,bool,int64,object,object,object,object,float64,float64,float64,float64,float64,object,object,str8,str16,float64,str16,object,int64,object
ZTF18abiwqjk,10,1759,58316.432106500026,59329.505381899886,True,True,2,None,None,None,None,58435.2652431,59329.505381899886,894.2401387998834,3.2398945,50.35809655,0.00012586500705121358,3.698168465613856e-05,bogus,stamp_classifier,0.38414162,correction_0.0.1,<Table length=1> obsid ra dec ... status public_avail_date deg deg ... object float64 float64 ... object object ------ ----------------- ----------------- ... -------- ------------------- 1864 3.072916666666667 50.43136111111111 ... archived 2002-01-09T00:00:00,1,obs_id ... ... ------ ... 1864 ...
ZTF18abiagdn,4,1770,58317.3802894,59329.505381899886,True,True,2,None,None,None,None,58435.2652431,59329.505381899886,894.2401387998834,3.23719265,50.10339005,0.0004404568140012446,0.00024515392103841997,bogus,stamp_classifier,0.6067099,correction_0.0.1,<Table length=1> obsid ra dec ... status public_avail_date deg deg ... object float64 float64 ... object object ------ ----------------- ----------------- ... -------- ------------------- 1864 3.072916666666667 50.43136111111111 ... archived 2002-01-09T00:00:00,1,obs_id ... ... ------ ... 1864 ...
ZTF18abiagbp,22,1592,58312.42182870023,59329.505381899886,True,True,11,-0.113813825,0.46666965,-0.23509538,0.553391,58435.2652431,59329.505381899886,894.2401387998834,3.092486972727272,50.5531224,0.0002128165176432332,9.867637001850209e-05,VS,stamp_classifier,0.73947304,correction_0.0.1,<Table length=1> obsid ra dec ... status public_avail_date deg deg ... object float64 float64 ... object object ------ ----------------- ----------------- ... -------- ------------------- 1864 3.072916666666667 50.43136111111111 ... archived 2002-01-09T00:00:00,1,obs_id ... ... ------ ... 1864 ...


In [ ]:
# To save the VOTable, run:
# filename = #Insert filename here
# writeto(maskedztf, filename)